<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 08</h1>

Let's revisit the first Jupyter notebook we ever saw in ECON 140R, back in class 1. As before, we will load up `wage2` from the Wooldridge repository, and this time we will formally examine omitted variable bias in a regression of the log wage on worker characteristics, where the treatment variable of interest often is years of education. 

Are years of education really as important for earnings as they appear to be in ordinary least squares (OLS) results? Or are there omitted variables?

<i>Does "ability" bias the coefficient on years of education higher than it would be in a randomized controlled trial?</i>

<h2>Learning Objectives</h2>

1. Run some ordinary least squares (OLS) regressions using `lm()` in __R__

2. Examine the <i>OVB formula</i> for this example, from pp 70-72 in <i>Mastering Metrics</i>

3. When an omitted variable $z$ is positively correlated with the outcome and positively correlated with a beneficial treatment, then omitting it creates an estimated effect of the treatment that is too positive

4. IQ or "ability" might be an omitted variable in a log wage regression on education: individuals with high ability complete more schooling and also are able to earn more. Is the true causal effect of schooling on earnings therefore smaller?

5. (Or is IQ a <i>bad control</i>? Or a bad something else? Read pp 213-214. It's probably a bad something else.)

<hr>

<h3>Blackburn and Neumark (1992)
</h3>
    
Let us examine something <b>omitted variable bias</b> using a useful repository of data from Jeffrey Wooldridge's excellent textbook, <i>Introductory Econometrics, a Modern Approach</i>. 
This appears as Example 9.3 on page 281 of the 6th edition, and it draws on a dataset provided by [Blackburn and Neumark (1992)](https://www-jstor-org.libproxy.berkeley.edu/stable/2118394) on monthly earnings and other characteristics among men observed in 1980. 

The data come from the [National Longitudinal Survey of Older and Young Men (NLS)](https://www.nlsinfo.org/content/cohorts/older-and-young-men), specifically the younger men who were originally surveyed in 1966 at ages 14-24, and subsequently up until 1981. The oldest of these men was born in 1942, and the youngest in 1952. Blackburn and Neumark look only at nonblack males, but in the `wage2` dataset, we have 120 men who identify as Black and 815 who did not. Oddly, in this extract from Wooldridge, who obtained it from the authors, we do not see measures of industry or occupation, even though those were the focus of the paper.

(This particular example is unfortunately not found within Florian Heiss's excellent R version of this book at [http://www.urfie.net/read/index.html](http://www.urfie.net/read/index.html))

<h3>"Ability" bias, measures, and a big asterisk</h3>

These data are from the same source examined in a classic paper by [Griliches (1977)](https://www-jstor-org.libproxy.berkeley.edu/stable/1913285), which is discussed on pp 213-214 of <i>Mastering Metrics</i>. In the 2015 version of applied labor economics, this examination by Griliches is "intriguing" but not "conclusive." As described in the textbook: while test scores are interesting, they have their own problems.

One problem is that test scores might be generated simultaneously with education, so they might represent something that evolves along with education, rather than something that causes both education and earnings. Another problem is that test scores probably do not do a better job that schooling of capturing nontraditional aspects of workers' productivity, like "Mick Jagger's charisma or Johan Harmenberg's perseverance."

As Angrist and Pischke make clear, 

<center><b>What we are about to do is interesting and worth doing, but it is outdated science.</b></center>

Please do not jump to the conclusion that "ability" bias is an important control in a log earnings regression, and IQ is the measure to control for. That might be what the 1977 vintage of knowledge suggests, but it is not what Angrist and Pischke think is most revealing. 

Still, it is useful to explore the test scores that we have in these NLS data. There are actually two measures available:

1. `IQ` is a standardized measure of "intelligence" on a once-familiar scale that may now seem quaint. It was collected from the schools attended by the survey participants

2. `KWW` is a score on a "Knowledge of the World of Work" test administered in 1966 by the study investigators 

A big conceptual problem is that `IQ` is measured contemporaneously with or before completion of education. The `KWW` measure may be cleaner because it was measured later in life. (But as reported by the authors cited above, the two measures tend to be fairly interchangeable in the econometrics.)

Helpfully, folks have dumped all Wooldridge's public datasets into an R package for us to use. Here is code that sets that up. Highlight the code snippet with your mouse or trackpad, and hit <tt>SHIFT+ENTER</tt>

In [ ]:
install.packages('wooldridge')

This command digs into that loaded package and retrieves part of it for our data:

In [ ]:
data(wage2, package='wooldridge')

There are several ways of probing what it is that we just loaded. One convenient function to call is <tt>head()<tt>:

In [ ]:
head(wage2)

&nbsp;

The variables have mnemonic names you can guess. Probably the strangest one is <tt>lwage</tt>, which appears at the far right of the results window (scroll right), and which is the <b>natural logarithm of the monthly wage</b>.

As we will see, in R the <tt>lm()</tt> function fits multivariate linear models conveniently. The syntax takes getting used to, but to estimate this model:
$$y = \alpha + \beta x + \gamma z + \epsilon$$
we  call this code:

<center><tt>lm(y ~ x + z)</tt></center>

Can you see the similarities?

Now bear with me. I am going to call <tt>lm()</tt> three times with different equations, and I am going to assign the output to new structures on the left hand side of the "gets" operator <tt><-</tt>

In [ ]:
shortreg <- lm(lwage ~ educ + exper + tenure 
               + married + south + urban + black, 
               data = wage2)

In [ ]:
longreg <- lm(lwage ~ educ + exper + tenure 
              + married + south + urban + black 
              + IQ, 
              data = wage2)

In [ ]:
auxreg <- lm(IQ ~ educ + exper + tenure 
             + married + south + urban + black, 
             data = wage2)

If you were to surround those three calls with parentheses, then R would immediately spit back something. Or you can wait and call <tt>summary()<tt>

In [ ]:
summary(shortreg)

&nbsp;

This and other output are what we are after. It takes some skill to decipher this, but you will learn to do it as effortlessly as I can. Look at the coefficient on <tt>educ</tt> in particular, rounded to the nearest thousandth:

In [ ]:
beta_short <- round( shortreg$coefficients["educ"], digits = 3)
beta_short

In words, it turns out that this is the <i>percentage increase</i> in the hourly wage for each additional year (unit) of education, because the $y$-variable is the natural logarithm of the wage. That's a 6.5% real return, which is not too shabby.

Now let's examine the long regression, where we have controlled for `IQ`:

In [ ]:
summary(longreg)

In [ ]:
beta_long <- round( longreg$coefficients["educ"], digits = 3)
beta_long

gamma_long <- round( longreg$coefficients["IQ"], digits = 3)
gamma_long

In words, what we have seen thus far:
* In the short regression without `IQ`, we found a rate of return to years of schooling of about 6.5% 
* In the long regression <i>with</i> `IQ`, we found a rate of return of about 5.4%
* In the long regression, we also see a rate of return to a unit of `IQ` equal to roughly 0.4% 

The final piece of this useful approach is the auxiliary regression, where we run the omitted variable `IQ` on the full set of regressors in the short regression:

In [ ]:
summary(auxreg)

In [ ]:
pi_1 <- round( auxreg$coefficients["educ"], digits = 3)
pi_1

There is a positive and statistically significant relationship $\pi$ between the omitted variable `IQ` and years of education. The omitted variable bias formula explains what we are seeing in the difference in rates of return to schooling between the short regression and the long regression:

$$
OVB = \beta^s - \beta^l = \pi_1 \times \gamma
$$

Below we can write out the left and right pieces of the math, first using rounded numbers, and then using the more precise originals:

In [ ]:
OVB_left = beta_short - beta_long
OVB_left

OVB_right = pi_1 * gamma_long
OVB_right

In [ ]:
OVB_left_p = shortreg$coefficients["educ"] - longreg$coefficients["educ"]
OVB_left_p

OVB_right_p = auxreg$coefficients["educ"] * longreg$coefficients["IQ"]
OVB_right_p

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>